In [21]:
import numpy as np
import tensorflow as tf

2024-04-03 14:09:06.072594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Exercise: Embedded Reber grammars *were used by Hochreiter and Schmidhuber in [their paper](https://scholar.google.com/scholar?q=Long+Short-Term+Memory+author%3ASchmidhuber) about LSTMs. They are artificial grammars that produce strings such as “BPBTSXXVPSEPE”. Check out Jenny Orr’s [nice introduction](https://willamette.edu/~gorr/classes/cs449/reber.html) to this topic, then choose a particular embedded Reber grammar (such as the one represented on Orr’s page), then train an RNN to identify whether a string respects that grammar or not. You will first need to write a function capable of generating a training batch containing about 50% strings that respect the grammar, and 50% that don’t.*

The link to the Jenny Orr webpage is dead, but I found another [link](http://christianherta.de/lehre/dataScience/machineLearning/neuralNetworks/reberGrammar.php) that can be used instead.

In [12]:
reber_grammar = [
    [("B", 1)],  # (state 0) = B => (state 1)
    [("T", 2), ("P", 3)],  # (state 1) = T => (state 2) or = P => (state 3)
    [("S", 2), ("X", 4)],  # (state 2) = S => (state 2) or = X => (state 4)
    [("T", 3), ("V", 5)],  # and so on ...
    [("X", 3), ("S", 6)],
    [("V", 6)],
    [("E", None)],  # (state 6 ) = E => (terminal state)
]

embed_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(reber_grammar, 4)],
    [(reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)],
]


def generate_string(grammar):
    output = []
    state = 0
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):  # deal with reber grammar
            production = generate_string(production)
        output.append(production)
    return "".join(output)

Let's generate a few string using the original Reber grammar.

In [13]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(reber_grammar), end=" ")

BTXXTTVVE BTSSXXTTTVVE BTXSE BPTVVE BTXSE BPVVE BPVVE BPVVE BTSXSE BPTVVE BTSSSSXSE BPVVE BPTVVE BPTVVE BTXXVVE BPTTTTTTTTVVE BPTVVE BPVVE BPTVVE BTXSE BPTVVE BTXXVVE BTSXXVVE BPVVE BPVVE 

Okay. Let's look at the embed Reber grammar.

In [14]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embed_reber_grammar), end=" ")

BTBPTTTVVETE BTBTSXXTTTVVETE BTBPVVETE BPBTXXVVEPE BPBPVVEPE BPBPVVEPE BPBTSXSEPE BPBTXXTTTTVVEPE BPBPVVEPE BPBTXSEPE BTBPTVVETE BTBPVVETE BTBTSSSSSSXSETE BTBPVVETE BPBPTVVEPE BTBPVVETE BPBTXXVVEPE BTBPTTVVETE BTBPVVETE BPBPVVEPE BPBPVVEPE BPBPVVEPE BTBTXSETE BPBPVVEPE BPBPVVEPE 

Now we need to generate strings that do not respect grammar. We could create a random string, but the task would be quite easy, so we instead will generate a valid string, and corrupt it by changing only one letter at random.

In [17]:
POSSIBLE_CHARS = "BEPSTXV"


def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    string = generate_string(grammar)
    index = np.random.randint(len(string))
    good_char = string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return string[:index] + bad_char + string[index + 1 :]

Let's look at a few corrupted strings.

In [18]:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embed_reber_grammar), end=" ")

BTBPETTVVETE BTBTSSXPVVETE BPBTXEEPE BPBPTTVVEXE BTBPTTTTTVVTTE BPTTXSEPE BTBPVVETS BTBTSSSSSSETE BPBPVVEPP BTBPVVTTE TTBPTTVVETE BXBPVVEPE BPBXVVEPE BEBTXSETE BPBPVVEXE BTBPTVVVTE BPBTXXTVVBPE BPBPVVEPV BTBTSSSPXXVVETE BTBTXXETTTVVETE BBBPTTVVETE BPBTPXSEPE BPEPVVEPE BTBTSXVTTVVETE BTBTSSSXSETV 

- We cannot feed string directly to an RNN, so we need to encode them somehow.
- One option would be one-hot encode each character. Another options is to use embeddings.
- We'll go with the second option, but since there are just a handful of characters, one-hot encoding can be used as well.
- For embeddings to work, we need to convert each string into a sequence of character IDs.

In [19]:
def string_to_ids(string, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in string]

In [20]:
string_to_ids("BTBPETTVVETE")

[0, 4, 0, 2, 1, 4, 4, 6, 6, 1, 4, 1]

In [28]:
def generate_dataset(size):
    good_strings = [
        string_to_ids(generate_string(embed_reber_grammar)) for _ in range(size // 2)
    ]
    corrupted_strings = [
        string_to_ids(generate_corrupted_string(embed_reber_grammar))
        for _ in range(size - size // 2)
    ]
    all_strings = good_strings + corrupted_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = tf.constant(
        [[1.0] for _ in range(len(good_strings))]
        + [[0.0] for _ in range(len(corrupted_strings))]
    )
    return X, y

In [32]:
tf.random.set_seed(42)
np.random.seed(42)

X_train, y_train = generate_dataset(10_000)
X_valid, y_valid = generate_dataset(2_000)

Let's look at the first training sequence:

In [33]:
X_train[0]

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 4, 0, 2, 4, 4, 4, 6, 6, 1, 4, 1], dtype=int32)>

And its correspond class

In [34]:
y_train[0]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [57]:
tf.random.set_seed(42)
np.random.seed(42)

embed_size = 5

model = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=[None], dtype=tf.int8, ragged=True),
        tf.keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embed_size),
        tf.keras.layers.GRU(30),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9, nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


2024-04-03 14:53:58.410547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-03 14:53:58.413387: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-03 14:53:58.415168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

313/313 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5130

2024-04-03 14:54:09.708521: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [2000,1]
	 [[{{node Placeholder/_1}}]]
2024-04-03 14:54:10.329702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-03 14:54:10.333947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '

313/313 [==============================] - 13s 28ms/step - loss: 0.6929 - accuracy: 0.5130 - val_loss: 0.6902 - val_accuracy: 0.4940
Epoch 2/20
313/313 [==============================] - 9s 29ms/step - loss: 0.6819 - accuracy: 0.5563 - val_loss: 0.6715 - val_accuracy: 0.4845
Epoch 3/20
313/313 [==============================] - 9s 29ms/step - loss: 0.6566 - accuracy: 0.5957 - val_loss: 0.6561 - val_accuracy: 0.5560
Epoch 4/20
313/313 [==============================] - 8s 25ms/step - loss: 0.6454 - accuracy: 0.5938 - val_loss: 0.6671 - val_accuracy: 0.6080
Epoch 5/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6374 - accuracy: 0.6051 - val_loss: 0.6476 - val_accuracy: 0.4650
Epoch 6/20
313/313 [==============================] - 6s 19ms/step - loss: 0.6272 - accuracy: 0.6163 - val_loss: 0.6466 - val_accuracy: 0.6060
Epoch 7/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6079 - accuracy: 0.6407 - val_loss: 0.6006 - val_accuracy: 0.6505
Epoch 8/2

Now we test our RNN on two tricky strings: the first one is bad while the second one is good. They only differ by the second ot last character. If the RNN get this right, it shows that it managed to understand that the second character is the same as the second to last character. This requires a fairly long short-term memory (which is why we used a GRU cell).

In [64]:
good_string = generate_string(embed_reber_grammar)
while len(good_string) < 20:
    good_string = generate_string(embed_reber_grammar)
bad_char = "T" if good_string[1] == "P" else "P"
bad_string = list(good_string)
bad_string[-2] = bad_char
bad_string = "".join(bad_string)
print(good_string, bad_string, sep="\n")
test_strings = [bad_string, good_string]

BTBTSSSSSXXTTTTTTTVVETE
BTBTSSSSSXXTTTTTTTVVEPE


In [65]:
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print()
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

1/1 [==============================] - 0s 26ms/step

Estimated probability that these are Reber strings:
BTBTSSSSSXXTTTTTTTVVEPE: 6.58%
BTBTSSSSSXXTTTTTTTVVETE: 99.94%


2024-04-03 14:57:15.186813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [2]
	 [[{{node Placeholder/_0}}]]
